In [1]:
import pandas as pd
from pyne.material import Material as m
from pyne import nucname
import numpy as np

/home/gwenchee/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.data is not yet QA compliant.
  return f(*args, **kwds)
/home/gwenchee/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.material is not yet QA compliant.
  return f(*args, **kwds)


In [11]:
names = [
        'assembly_id',
        'reactor_id',
        'reactor_type',
        'initial_uranium_kg',
        'initial_enrichment',
        'discharge_burnup',
        'discharge_date',
        'discharge_time',
        'total_assembly_decay_heat_kw',
        'name',
        'evaluation_date',
        'total_mass_g',
        'total_radioactivity_curies']
df = pd.read_csv('SNF_nuclide_1yr_Cooling.dat',
                 sep='\s+',
                 index_col=False,
                 names=names)

In [12]:
keep = df[['assembly_id','reactor_id','reactor_type','discharge_date','total_assembly_decay_heat_kw','name','total_mass_g']]

In [13]:
#BWRs 
BWR = keep.loc[keep['reactor_type'] == 'BWR']
# keep first BWR 
BWR_1 = keep.loc[keep['reactor_id'] == 122]
# keep first assembly
BWR_2 = keep.loc[keep['assembly_id'] == 254897]
heat = keep.groupby((df["assembly_id"] != df["assembly_id"].shift()).cumsum().values).first()
heat = heat[['assembly_id','total_assembly_decay_heat_kw']]

In [14]:
def decay_assembly(assembly_id): 
    assem = keep.loc[keep['assembly_id'] == assembly_id]
    compdict = {}
    for index, row in assem.iterrows():
        if row['assembly_id'] == assembly_id: 
            nuc = row['name']
            mass = row['total_mass_g']
            if nuc == 'ag-108m' or nuc == 'am-242m' or nuc == 'ag-110m': # meta stable nucs dont work?
                compdict = compdict 
            else:
                compdict[nucname.id(nuc)] = mass
    new_m = m(compdict,mass = -1)
    decayheat = new_m.decay_heat()
    decayheat_total = 0
    for key,val in decayheat.items(): 
        decayheat_total += val*10**3
    
    return decayheat_total 

In [15]:
heat

,assembly_id,total_assembly_decay_heat_kw
1,254897,0.801130
2,254898,0.801420
3,254899,0.801013
4,254900,0.800920
5,254901,0.797371
6,254902,0.798022
7,254903,0.797828
8,254904,0.797845
9,254905,0.790827
10,254906,0.791610


In [ ]:
n = 244895
newheat = heat.head(n)
calc_heat = np.zeros(n)
perc_diff = np.zeros(n)
i = 0
LOG_EVERY_N = 1000
for index, row in newheat.iterrows():
    calc_heat[i] = decay_assembly(row['assembly_id'])
    perc_diff[i] = calc_heat[i]/row['total_assembly_decay_heat_kw']*100
    if (i % LOG_EVERY_N) == 0:
        print('LOG',i)
    i +=1
newheat['calc_heat'] = calc_heat
newheat['perc_diff'] = perc_diff

LOG 0
LOG 1000
LOG 2000
LOG 3000


In [ ]:
newheat

In [ ]:
# To store once created
newheat.to_pickle(decayheat)

In [ ]:
# To load picked files into this notebook
newheat = pd.read_pickle(decayheat)